# Intro to spark df api

### Import the SparkSession object

In [1]:
from os import environ
from pyspark.sql.session import SparkSession

### Creating a spark session

In [2]:
spark = SparkSession.builder.appName('df_basics').getOrCreate()

### Default path to the file folder

In [3]:
# file_path = "file:///var/lib/jupyter/data/"
file_path = "file:///"+environ['DATA_LAKE']
df = spark.read.json(file_path+"people.json")
# import os
# os.getcwd()

### Show data frame in spark

In [4]:
df.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



### Print schema details

In [5]:
df.printSchema()

root
 |-- age: long (nullable = true)
 |-- name: string (nullable = true)



### List all columns

In [6]:
df.columns

['age', 'name']

### Describe data frame

In [7]:
df.describe()

DataFrame[summary: string, age: string, name: string]

### See the description (automatically shows the statistics)

In [8]:
df.describe().show()

+-------+------------------+-------+
|summary|               age|   name|
+-------+------------------+-------+
|  count|                 2|      3|
|   mean|              24.5|   null|
| stddev|7.7781745930520225|   null|
|    min|                19|   Andy|
|    max|                30|Michael|
+-------+------------------+-------+



### See the column object

In [9]:
df['age']

Column<'age'>

### Select a particular column value and show it

In [10]:
df.select('age').show()

+----+
| age|
+----+
|null|
|  30|
|  19|
+----+



### Select a particular column value type i.e DataFrame type

In [11]:
type(df.select('age'))

pyspark.sql.dataframe.DataFrame

### List the top two rows as list of rows

In [12]:
df.head(2)

[Row(age=None, name='Michael'), Row(age=30, name='Andy')]

### See the first row element type 

In [13]:
print(type(df.head(2)[0]), df.head(2)[0])

<class 'pyspark.sql.types.Row'> Row(age=None, name='Michael')


### Select multiple columns

In [14]:
df.select(['age','name']).show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



### Add a new column to the datafram using withColumn (NOT A INPLACE OPERATION)

In [15]:
df.withColumn('half_age', df['age']/2).show()

+----+-------+--------+
| age|   name|half_age|
+----+-------+--------+
|null|Michael|    null|
|  30|   Andy|    15.0|
|  19| Justin|     9.5|
+----+-------+--------+



### Rename a column

In [16]:
df.withColumnRenamed('age', 'new_age').show()

+-------+-------+
|new_age|   name|
+-------+-------+
|   null|Michael|
|     30|   Andy|
|     19| Justin|
+-------+-------+



### Create a view from a data frame

In [17]:
df.createOrReplaceTempView('people')

### Use spark sql to fetch the data from the view

In [18]:
result = spark.sql('SELECT * FROM PEOPLE')

In [19]:
result.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



### More complex sql (using rank())

In [20]:
new_results = spark.sql('SELECT name, age, rank() over(partition by name order by age desc) as rn FROM PEOPLE WHERE AGE = 30')

In [21]:
new_results.show()

+----+---+---+
|name|age| rn|
+----+---+---+
|Andy| 30|  1|
+----+---+---+



In [22]:
spark.stop()